# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [68]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer

## lda
from gensim import corpora
import gensim

# Counter
from collections import Counter

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ryansorkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ryansorkin/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
pharma = doj[doj['id'] == "17-1204"].contents.astype(str).iloc[0]
pharma

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
# A
# Proprocess tect and remove punct and digits
def preprocess_text(text):
    # Tokenize
    words = text.split()
    
    # convert to lowercase
    cleaned_words = [word.lower() for word in words if word.isalpha()]
    
    #Join back 
    cleaned_text = ' '.join(cleaned_words)
    
    return cleaned_text

# Call function
cleaned_pharma = preprocess_text(pharma)

In [5]:
# B

## Tokenize
words = word_tokenize(cleaned_pharma)

## Tag words
pos_tags = pos_tag(words)

In [6]:
## your code here for part of speech tagging
# Only adjectvices
adjectives = [word for word, pos in pos_tags if pos in ['JJ', 'JJR', 'JJS']]

adjective_counts = Counter(adjectives)
most_common_adjectives = adjective_counts.most_common(5)

#Create df
df = pd.DataFrame(most_common_adjectives, columns=['Adjective', 'Count'])

df


,Adjective,Count
0,former,8
1,special,7
2,nationwide,4
3,criminal,4
4,postal,4


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [7]:
## your code here for part A
spacy_pharma = nlp(pharma)

named_entities = [(ent.text, ent.label_) for ent in spacy_pharma.ents]
named_entities

## FROM SLIDES
# for ent in spacy_pharma.ents:
#     print("Entity: " + ent.text + 
#          "; Ner Tag: " + ent.label_)


[('Insys Therapeutics Inc.', 'ORG'),
 ('today', 'DATE'),
 ('Fentanyl', 'PERSON'),
 ('More than 20,000', 'CARDINAL'),
 ('Americans', 'NORP'),
 ('last year', 'DATE'),
 ('millions', 'CARDINAL'),
 ('Jeff Sessions', 'PERSON'),
 ('This Justice Department', 'ORG'),
 ('Trump', 'PERSON'),
 ('American', 'NORP'),
 ('”John N. Kapoor', 'PERSON'),
 ('74', 'DATE'),
 ('Phoenix', 'GPE'),
 ('Ariz.', 'GPE'),
 ('the Board of Directors', 'ORG'),
 ('Insys', 'ORG'),
 ('this morning', 'TIME'),
 ('Arizona', 'GPE'),
 ('RICO', 'LAW'),
 ('Kapoor', 'PERSON'),
 ('Executive', 'ORG'),
 ('Board', 'ORG'),
 ('Insys', 'ORG'),
 ('Phoenix', 'GPE'),
 ('today', 'DATE'),
 ('U.S.', 'GPE'),
 ('District Court', 'ORG'),
 ('Boston', 'GPE'),
 ('a later date', 'DATE'),
 ('today', 'DATE'),
 ('Boston', 'GPE'),
 ('Insys', 'ORG'),
 ('December 2016.The', 'DATE'),
 ('Kapoor', 'GPE'),
 ('Michael L. Babich', 'PERSON'),
 ('40', 'DATE'),
 ('Scottsdale', 'GPE'),
 ('Ariz.', 'GPE'),
 ('Alec Burlakoff', 'PERSON'),
 ('42', 'DATE'),
 ('Charlotte', 

In [8]:
## your code here for part B
laws = set([ent[0] for ent in named_entities if ent[1] == "LAW"])
print(laws)

{'RICO', 'the Controlled Substances Act'}


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

## your code here 
The RICO act is a U.S. federal law that allows for the prosecution and civil penalties against individuals and entities engaged in organized criminal activities. This relates to our case because this case revolves around a very high up individual who was involved in using his power to make doctors prescribe his drug more often to patients who may not have needed it. 

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## your code here
sentence_entities = [ent[0] for ent in named_entities if ent[1] == "DATE" and re.search(r'\byear[s]?\b', ent[0], flags=re.I)]
print(sentence_entities)

['last year', '20 years', 'three years', 'five years', 'three years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
# ###MY WAY
## your code here
for date in sentence_entities:
    sentences = re.findall(r'[^.]*' + re.escape(date) + r'[^.]*\.', pharma)
    
    for sentence in sentences:
        print(sentence)
        


 "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
  The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.
  The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.
The charges o

In [11]:
# ELI WAY
[sentence.text for sentence in spacy_pharma.sents
if any(length in sentence.text for length in sentence_entities)]


['"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.',
 'The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.\xa0 ',
 'The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']

In [12]:
# Append Outputs 
sentences_length = []
for sentence in spacy_pharma.sents:
    for length in sentence_entities:
        if re.search(length, sentence.text):
            sentences_length.append(sentence.text)
            break 

print(sentences_length)

['"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.', 'The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.\xa0 ', 'The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']


# Sentence Length: 
With the conspiracy to commit RICO coincided with conspiracy to violate the Anti-Kickback law, a sentence maximum of 25 years could be possible

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [13]:
## your code here for subsetting
doj_subset = doj[doj['topics_clean'].isin(['Civil Rights', 'Hate Crimes', 'Project Safe Childhood'])]
doj_subset.shape


(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [14]:
## your code here to define function

def analysis(press_release):
    S = SentimentIntensityAnalyzer()
    
    spacy = nlp(press_release)    

    #     Find entities
    named_ents = [ent.text for ent in spacy.ents]

    #     Entitites pattern
    named_ents_pattern = '|'.join(map(re.escape, named_ents))
    
    #get rid of entities 
    clean_release = re.sub(named_ents_pattern, '', press_release)
    
    # create polarity score
    score = S.polarity_scores(clean_release)
    
    return score

In [15]:
## your code here executing the function
result = doj_subset.contents.apply(lambda x: analysis(x))

print(result)

77        {'neg': 0.216, 'neu': 0.73, 'pos': 0.054, 'compound': -0.9931}
155      {'neg': 0.148, 'neu': 0.774, 'pos': 0.078, 'compound': -0.9292}
157       {'neg': 0.098, 'neu': 0.821, 'pos': 0.081, 'compound': -0.738}
162      {'neg': 0.134, 'neu': 0.775, 'pos': 0.091, 'compound': -0.8981}
168      {'neg': 0.195, 'neu': 0.757, 'pos': 0.048, 'compound': -0.9864}
                                      ...                               
13002    {'neg': 0.166, 'neu': 0.765, 'pos': 0.069, 'compound': -0.9689}
13032      {'neg': 0.092, 'neu': 0.798, 'pos': 0.11, 'compound': 0.5667}
13034     {'neg': 0.179, 'neu': 0.72, 'pos': 0.101, 'compound': -0.9635}
13068    {'neg': 0.146, 'neu': 0.747, 'pos': 0.107, 'compound': -0.9786}
13081      {'neg': 0.167, 'neu': 0.8, 'pos': 0.033, 'compound': -0.9931}
Name: contents, Length: 717, dtype: object


C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [16]:
## your code here
sentiment_scores = [analysis(text) for text in doj_subset['contents']]

In [42]:
doj_subset['negative'] = [score['neg'] for score in sentiment_scores]
doj_subset['positive'] = [score['pos'] for score in sentiment_scores]
doj_subset['neutral'] = [score['neu'] for score in sentiment_scores]
doj_subset['compoundX'] = [score['compound'] for score in sentiment_scores]

doj_subset_wscore = doj_subset.sort_values(by='negative', ascending=False)
doj_subset_wscore[['id', 'contents', 'negative']].head(2)

,id,contents,negative
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.362
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.323


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [43]:
## agg and find the mean compound score by topic
doj_subset_wscore.groupby('topics_clean').agg({'compoundX' : 'mean'})

,compoundX
topics_clean,
Civil Rights,-0.095695
Hate Crimes,-0.939660
Project Safe Childhood,-0.645745


My assumption would be that when talking about civil rights and/or hate crimes, the langauge used will be much more deragatory to explain what has happened or what was said.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [44]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [45]:
## your code defining a text processing function
def function2(content):
    snow_stemmer = SnowballStemmer(language='english')

    #   stopwords
    list_stopwords = stopwords.words('english')
#   combine
    list_stopwords_new = custom_doj_stopwords + list_stopwords

    #   to lower
    lower = content.lower()
    
    nostopwords = [word for word in wordpunct_tokenize(lower)
                  if word not in list_stopwords_new]
    preprocess = [snow_stemmer.stem(word) for word in nostopwords
                 if word.isalpha() and len(word) > 4]
    full = " ".join(preprocess)
    
    return(full)

In [46]:
## your code executing the function
doj_subset_wscore['processed_text'] = doj_subset_wscore.contents.apply(lambda x: function2(x))

In [47]:
## your code showing the examples
examples = doj_subset_wscore[doj_subset_wscore['id'].isin(["16-718","16-217"])]
examples[['id', 'contents', 'processed_text']]

,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",count indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison cruel unusu punish sturdiv charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three convict marsher sturdiv maximum sentenc year prison excess forc charg three offic face year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi mulhaus crimin section marsher indict
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectiv

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [48]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns = vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [49]:
# your code here
dtm = create_dtm(doj_subset_wscore['processed_text'], doj_subset_wscore[['id', 'compoundX', 'topics_clean']])
dtm.head()

,index,id,compoundX,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zachari,zamora,zealand,zealous,zeeman,zionism,zobel,zone,zunggeemog,zwengel
0,329,14-248,-0.9951,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,572,13-312,-0.9983,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,11593,16-718,-0.9964,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,-0.9986,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11248,10-1194,-0.9989,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# function to get top 10
def get_topwords(data, n):
   
    # Sum the counts of each word (excluding metadata columns)
    word_sums = data.drop(columns = ["index"] + metadata_cols).sum(axis=0)
    
    # Return the top n words
    return word_sums.nlargest(n)


In [63]:
# B
top_5_percentile = dtm['compoundX'].quantile(0.95)
dtm1 = dtm[dtm['compoundX'] >= top_5_percentile]
get_topwords(dtm1, 10)

agreement     175
enforc        130
disabl        115
state         115
communiti     111
ensur         109
polic          93
settlement     86
student        85
servic         84
dtype: int64

In [64]:
# C
bot_5_percentile = dtm['compoundX'].quantile(0.05)
dtm2 = dtm[dtm['compoundX'] <= bot_5_percentile]
get_topwords(dtm2, 10)

assault     208
crime       178
victim      166
defend      131
offic       121
conspir     116
sentenc     109
charg       104
american    103
anderson     94
dtype: int64

In [67]:
# D
for topic in dtm['topics_clean'].unique():
    topic_subset = dtm[dtm['topics_clean'] == topic]
    
    print(f"\nTop 10 words for press releases in topic '{topic}':")
    print(get_topwords(topic_subset, 10))


Top 10 words for press releases in topic 'Hate Crimes':
victim      591
crime       557
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
african     370
dtype: int64

Top 10 words for press releases in topic 'Civil Rights':
offic        637
hous         633
discrimin    616
enforc       544
disabl       532
feder        479
violat       477
state        452
court        414
agreement    411
dtype: int64

Top 10 words for press releases in topic 'Project Safe Childhood':
child          1022
exploit         701
sexual          572
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
sentenc         355
dtype: int64


## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [71]:
# your code here
#convert text data to a document-term matrix
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

# use contents because we want preprocessed text 
dtm = count_vectorizer.fit_transform(doj_subset_wscore['contents'])

n_topics = 3
# Create LDA
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)

# fit model
lda.fit(dtm)

LatentDirichletAllocation(n_components=3, random_state=0)

In [74]:
# Run model
n_words = 15
feature_names = count_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_words - 1:-1]]))

Topic #1:
child district case exploitation safe project childhood pornography children sexual criminal ceos guilty victims trial
Topic #2:
rights civil housing discrimination district disabilities act agreement said court gov ada settlement fair federal
Topic #3:
rights civil district fbi federal said case guilty hate law victim trial office prison indictment


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [77]:
## your code here to get doc-level topic probabilities
topic_probabilities = lda.transform(dtm)
assert len(topic_probabilities) == len(doj_subset_wscore)

In [102]:
## your code here to add those topic probabilities to the dataframe
doj_subset_wscore['topic_1'] = topic_probabilities[:, 0]
doj_subset_wscore['topic_2'] = topic_probabilities[:, 1]
doj_subset_wscore['topic_3'] = topic_probabilities[:, 2]

# Make things easier and add topic 
doj_subset_wscore["top_topic"] = doj_subset_wscore[["topic_1_prob", "topic_2_prob", "topic_3_prob"]].idxmax(axis=1)

In [103]:
# Labels
for label in doj_subset_wscore['topics_clean'].unique():
    print(f"Breakdown for {label}:")
    breakdown = pd.crosstab(doj_subset_wscore[doj_subset_wscore['topics_clean'] == label]['top_topic'], columns="percentage", normalize=True) * 100
    print(breakdown, "\n")

Breakdown for Hate Crimes:
col_0         percentage
top_topic               
topic_3_prob       100.0 

Breakdown for Civil Rights:
col_0         percentage
top_topic               
topic_1_prob    0.327869
topic_2_prob   65.573770
topic_3_prob   34.098361 

Breakdown for Project Safe Childhood:
col_0         percentage
top_topic               
topic_1_prob    99.39759
topic_2_prob     0.60241 



In the case of 17-1235, it falls under the category of Civil Rights, but our model believed it was a hate crime. This could be because of words associated with violence within the text, suggesting the possibility of a hate crime. Case 17-379 was also a Civil Rights case and has been categorized by our model as a hate crime. This could be due to the occurrence of terms such as "imprisonment" and "prosecutor" within the text, which align with the terminology associated with our representation of the hate crimes topic.







# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [91]:
## your code here 
def create_bigram_onedoc(processed_text):
    words = processed_text.split()
    bigrams = [f'{words[i]}_{words[i+1]}' for i in range(len(words)-1)]
    return ' '.join(bigrams)

doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)
doj_subset_wscore[doj_subset_wscore['id'] == "16-217"][["id","processed_text","processed_text_bigrams"]]

,id,processed_text,processed_text_bigrams
6727,16-217,reach comprehens settlement agreement miami miami polic resolv offic involv shoot offic announc princip deputi general vanita gupta wifredo ferrer southern florida settlement approv miami commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu identifi pattern practic excess forc offic involv shoot violat fourth amend constitut complianc settlement monitor independ review former tampa florida polic chief castor settlement agreement implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quick investig offic involv shoot occur measur includ settlement repres renew commit miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform princip deputi general gupta agreement strengthen relationship communiti serv improv account offic weapon unlaw provid communiti particip enforc agreement today agreement result joint effort miami ensur miami polic continu effort communiti protect sacr constitut citizen ferrer oversight communic agreement seek perman posit chang former chief orosa chief llane applaud commiss settlement agreement build import reform implement sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_miami miami_miami miami_polic polic_resolv resolv_offic offic_involv involv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_offic offic_involv involv_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_offic offic_involv involv_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_castor castor_settlement settlement_agreement agreement_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_offic offic_involv involv_shoot shoot_effect effect_quick quick_investig investig_offic offic_involv involv_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_today today_agreement agreement_result result_joint joint_effort effort_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_communiti communiti_protect protect_sacr sacr_constitut constitut_citizen citizen_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_applaud applaud_commiss commiss_settlement settlement_agreement agreement_build build_import import_reform reform_implement implement_sinc sinc_issu issu_find find_includ includ_conduct conduct_attorney attorney_staff staff_special speci

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [96]:
# your code here
bigrams_dtm = create_dtm(list_of_strings = doj_subset_wscore.processed_text_bigrams,
                        metadata = doj_subset_wscore[["id","compoundX","topics_clean"]])
# Shapes
dtm.shape
bigrams_dtm.shape

(717, 6841)

(717, 65572)

The bigrams df is going to be much bigger because you aren't investigating solo words but how words interacts with other words now. Also, words are going to be on there more times if they are commonly paired up with many other words

In [97]:
bigrams_dtm.groupby("topics_clean").apply(lambda x: get_topwords(x, 10))

topics_clean                                
Civil Rights            deputi_general          221
                        princip_deputi          221
                        vanita_gupta            202
                        general_vanita          199
                        unit_state              156
                        nation_origin           143
                        consent_decre           128
                        announc_today           122
                        settlement_agreement     94
                        polic_offic              82
Hate Crimes             african_american        367
                        plead_guilti            275
                        year_prison             147
                        special_agent           124
                        racial_motiv            114
                        thoma_perez             111
                        perez_general            95
                        unit_state               88
                   

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 